In [ ]:
# Установка зависимостей
%pip install torch transformers nltk networkx pandas numpy pymupdf

In [4]:
import os
import glob
import re
import json
import collections
import itertools
from typing import Dict, List, Tuple, Any, Optional
import logging

# Для работы с PDF
import fitz  # PyMuPDF

# Для машинного обучения
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from transformers import AutoConfig
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Для построения графов
import networkx as nx

# Для базовой обработки текста
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

# Настройка логгирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class PDFTextExtractor:
    """
    Компонент для извлечения текста из PDF документов с исключением
    изображений, формул и таблиц согласно техническому заданию.
    """
    
    def __init__(self):
        # Паттерны для исключения нетекстового содержимого
        self.figure_patterns = [
            r'Figure\s+\d+[.:]\s*.*',
            r'Fig\.\s*\d+[.:]\s*.*',
            r'Table\s+\d+[.:]\s*.*',
            r'Tab\.\s*\d+[.:]\s*.*'
        ]
        
        # Паттерны для математических формул
        self.formula_patterns = [
            r'\$.*?\$',  # LaTeX inlineTeX display math
            r'\\\[.*?\\\]',  # LaTeX display math
            r'\\begin\{.*?\}.*?\\end\{.*?\}',  # LaTeX environments
        ]
    
    def extract_text_from_pdf(self, pdf_path: str) -> str:
        """
        Извлекает текст из PDF файла, исключая изображения, формулы и таблицы.
        
        Args:
            pdf_path: Путь к PDF файлу
            
        Returns:
            str: Извлеченный и очищенный текст
        """
        try:
            doc = fitz.open(pdf_path)
            full_text = ""
            
            for page_num in range(len(doc)):
                page = doc.load_page(page_num)
                
                # Извлекаем только текст, исключая изображения
                text = page.get_text("text")
                
                # Фильтруем нетекстовое содержимое
                cleaned_text = self._filter_non_text_content(text)
                full_text += cleaned_text + "\n"
            
            doc.close()
            logger.info(f"Extracted text from {pdf_path}: {len(full_text)} characters")
            return full_text
            
        except Exception as e:
            logger.error(f"Error extracting text from {pdf_path}: {str(e)}")
            return ""
    
    def _filter_non_text_content(self, text: str) -> str:
        """
        Фильтрует ссылки на рисунки, таблицы и формулы из текста.
        
        Args:
            text: Исходный текст
            
        Returns:
            str: Отфильтрованный текст
        """
        # Удаляем ссылки на рисунки и таблицы
        for pattern in self.figure_patterns:
            text = re.sub(pattern, '', text, flags=re.IGNORECASE | re.MULTILINE)
        
        # Удаляем математические формулы
        for pattern in self.formula_patterns:
            text = re.sub(pattern, '', text, flags=re.DOTALL)
        
        # Удаляем избыточные пробелы и переносы строк
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'\n\s*\n', '\n', text)
        
        return text.strip()


class TextPreprocessor:
    """
    Компонент для предварительной обработки текста с использованием NLTK
    вместо spaCy, включая токенизацию и лемматизацию.
    """
    
    def __init__(self):
        # Загружаем необходимые ресурсы NLTK
        try:
            nltk.data.find('tokenizers/punkt')
        except LookupError:
            nltk.download('punkt')
        
        try:
            nltk.data.find('corpora/wordnet')
        except LookupError:
            nltk.download('wordnet')
        
        try:
            nltk.data.find('corpora/omw-1.4')
        except LookupError:
            nltk.download('omw-1.4')
        
        from nltk.stem import WordNetLemmatizer
        self.lemmatizer = WordNetLemmatizer()
        
        # Стоп-слова для разных языков
        self.stop_words = {
            'en': {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by'},
            'technical': {'fig', 'figure', 'table', 'tab', 'eq', 'equation', 'ref', 'reference'}
        }
    
    def preprocess_text(self, text: str) -> Dict[str, Any]:
        """
        Выполняет предварительную обработку текста.
        
        Args:
            text: Исходный текст
            
        Returns:
            Dict: Результат обработки с сегментированными предложениями и токенами
        """
        # Сегментация на предложения
        sentences = self._segment_sentences(text)
        
        # Обработка каждого предложения
        processed_sentences = []
        for sentence in sentences:
            if len(sentence.strip()) > 20:  # Фильтруем слишком короткие предложения
                tokens = self._tokenize_and_lemmatize(sentence)
                if len(tokens) >= 3:  # Минимум 3 токена в предложении
                    processed_sentences.append({
                        'original': sentence,
                        'tokens': tokens,
                        'length': len(tokens)
                    })
        
        return {
            'sentences': processed_sentences,
            'total_sentences': len(processed_sentences),
            'total_tokens': sum(len(s['tokens']) for s in processed_sentences)
        }
    
    def _segment_sentences(self, text: str) -> List[str]:
        """Сегментирует текст на предложения."""
        sentences = sent_tokenize(text)
        return [s.strip() for s in sentences if len(s.strip()) > 10]
    
    def _tokenize_and_lemmatize(self, sentence: str) -> List[str]:
        """
        Токенизирует предложение и применяет лемматизацию.
        
        Args:
            sentence: Предложение для обработки
            
        Returns:
            List[str]: Список лемматизированных токенов
        """
        # Токенизация
        tokens = word_tokenize(sentence.lower())
        
        # Фильтрация и лемматизация
        processed_tokens = []
        for token in tokens:
            # Оставляем только буквенно-цифровые токены длиной > 2
            if re.match(r'^[a-zA-Z0-9]{3,}$', token):
                # Проверяем, что токен не является стоп-словом
                if not self._is_stop_word(token):
                    # Лемматизация
                    lemmatized = self.lemmatizer.lemmatize(token)
                    processed_tokens.append(lemmatized)
        
        return processed_tokens
    
    def _is_stop_word(self, token: str) -> bool:
        """Проверяет, является ли токен стоп-словом."""
        return (token in self.stop_words['en'] or 
                token in self.stop_words['technical'])


class BERTEntityExtractor:
    """
    Компонент для извлечения именованных сущностей с использованием BERT.
    Формирует вершины Knowledge Graph.
    """
    
    def __init__(self, model_name: str = "dbmdz/bert-large-cased-finetuned-conll03-english"):
        self.model_name = model_name
        self.confidence_threshold = 0.8
        
        # Инициализируем BERT NER pipeline
        try:
            self.ner_pipeline = pipeline(
                "ner",
                model=model_name,
                tokenizer=model_name,
                aggregation_strategy="simple",
                device=0 if torch.cuda.is_available() else -1
            )
            logger.info(f"Loaded BERT NER model: {model_name}")
        except Exception as e:
            logger.error(f"Error loading BERT model: {str(e)}")
            self.ner_pipeline = None
        
        # Специфичные для металлургии типы сущностей
        self.metallurgy_entity_types = {
            'MATERIAL': ['copper', 'steel', 'iron', 'aluminum', 'alloy', 'metal'],
            'CHEMICAL': ['carbon', 'oxygen', 'sulfur', 'silicon', 'manganese'],
            'EQUIPMENT': ['furnace', 'converter', 'ladle', 'casting', 'rolling'],
            'PROCESS': ['smelting', 'refining', 'annealing', 'quenching', 'tempering']
        }
    
    def extract_entities(self, sentences: List[Dict]) -> List[Dict]:
        """
        Извлекает именованные сущности из предложений.
        
        Args:
            sentences: Список обработанных предложений
            
        Returns:
            List[Dict]: Список извлеченных сущностей
        """
        if not self.ner_pipeline:
            return []
        
        all_entities = []
        
        for sent_data in sentences:
            sentence = sent_data['original']
            
            try:
                # BERT NER
                bert_entities = self.ner_pipeline(sentence)
                
                # Фильтруем по уверенности
                filtered_entities = [
                    entity for entity in bert_entities 
                    if entity['score'] >= self.confidence_threshold
                ]
                
                # Добавляем доменную специфику
                domain_entities = self._add_domain_entities(sentence)
                
                # Объединяем результаты
                sentence_entities = self._merge_entities(filtered_entities, domain_entities)
                
                for entity in sentence_entities:
                    entity['sentence'] = sentence
                    entity['sentence_id'] = hash(sentence)
                
                all_entities.extend(sentence_entities)
                
            except Exception as e:
                logger.warning(f"Error processing sentence for NER: {str(e)}")
                continue
        
        logger.info(f"Extracted {len(all_entities)} entities")
        return all_entities
    
    def _add_domain_entities(self, sentence: str) -> List[Dict]:
        """
        Добавляет доменно-специфичные сущности для металлургии.
        
        Args:
            sentence: Предложение для анализа
            
        Returns:
            List[Dict]: Список доменных сущностей
        """
        domain_entities = []
        sentence_lower = sentence.lower()
        
        for entity_type, keywords in self.metallurgy_entity_types.items():
            for keyword in keywords:
                if keyword in sentence_lower:
                    # Находим точные позиции
                    start_pos = sentence_lower.find(keyword)
                    if start_pos != -1:
                        domain_entities.append({
                            'word': keyword,
                            'entity_group': entity_type,
                            'score': 0.9,  # Высокая уверенность для доменных терминов
                            'start': start_pos,
                            'end': start_pos + len(keyword)
                        })
        
        return domain_entities
    
    def _merge_entities(self, bert_entities: List[Dict], domain_entities: List[Dict]) -> List[Dict]:
        """
        Объединяет сущности из BERT и доменно-специфичные сущности.
        
        Args:
            bert_entities: Сущности от BERT
            domain_entities: Доменно-специфичные сущности
            
        Returns:
            List[Dict]: Объединенный список сущностей
        """
        all_entities = bert_entities + domain_entities
        
        # Удаляем дубликаты по позиции
        unique_entities = []
        seen_positions = set()
        
        for entity in all_entities:
            pos_key = (entity.get('start', 0), entity.get('end', 0))
            if pos_key not in seen_positions:
                seen_positions.add(pos_key)
                unique_entities.append(entity)
        
        return unique_entities


class BiLSTMCRFRelationExtractor:
    """
    Компонент для извлечения отношений между сущностями 
    с использованием BiLSTM-CRF архитектуры.
    """
    
    def __init__(self, embedding_dim: int = 768, hidden_dim: int = 256):
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        # Типы отношений согласно ТЗ
        self.relation_types = {
            'FUNCTIONAL': [
                'causes', 'produces', 'affects', 'transforms', 'processes',
                'melts', 'cools', 'heats', 'combines', 'separates'
            ],
            'HIERARCHICAL': [
                'contains', 'part_of', 'composed_of', 'includes', 'consists_of',
                'made_of', 'derived_from', 'type_of', 'category_of'
            ]
        }
        
        # Инициализируем простую модель на основе паттернов
        self._init_pattern_based_extractor()
    
    def _init_pattern_based_extractor(self):
        """Инициализирует экстрактор отношений на основе паттернов."""
        # Паттерны для функциональных отношений (глагольные связи)
        self.functional_patterns = [
            r'(\w+)\s+(causes?|results?\s+in|leads?\s+to|produces?)\s+(\w+)',
            r'(\w+)\s+(transforms?|converts?|changes?)\s+(\w+)',
            r'(\w+)\s+(melts?|heats?|cools?|processes?)\s+(\w+)',
            r'(\w+)\s+(affects?|influences?|impacts?)\s+(\w+)'
        ]
        
        # Паттерны для иерархических отношений
        self.hierarchical_patterns = [
            r'(\w+)\s+(contains?|includes?|consists?\s+of)\s+(\w+)',
            r'(\w+)\s+(is\s+)?made\s+of\s+(\w+)',
            r'(\w+)\s+(is\s+a\s+)?type\s+of\s+(\w+)',
            r'(\w+)\s+(part\s+of|component\s+of)\s+(\w+)'
        ]
    
    def extract_relations(self, sentences: List[Dict], entities: List[Dict]) -> List[Dict]:
        """
        Извлекает отношения между сущностями из предложений.
        
        Args:
            sentences: Список обработанных предложений
            entities: Список извлеченных сущностей
            
        Returns:
            List[Dict]: Список извлеченных отношений
        """
        relations = []
        
        # Группируем сущности по предложениям
        entities_by_sentence = {}
        for entity in entities:
            sent_id = entity.get('sentence_id')
            if sent_id not in entities_by_sentence:
                entities_by_sentence[sent_id] = []
            entities_by_sentence[sent_id].append(entity)
        
        # Извлекаем отношения для каждого предложения
        for sent_data in sentences:
            sentence = sent_data['original']
            sent_id = hash(sentence)
            
            if sent_id in entities_by_sentence:
                sent_entities = entities_by_sentence[sent_id]
                sent_relations = self._extract_sentence_relations(sentence, sent_entities)
                relations.extend(sent_relations)
        
        logger.info(f"Extracted {len(relations)} relations")
        return relations
    
    def _extract_sentence_relations(self, sentence: str, entities: List[Dict]) -> List[Dict]:
        """
        Извлекает отношения из одного предложения.
        
        Args:
            sentence: Предложение
            entities: Сущности в предложении
            
        Returns:
            List[Dict]: Отношения в предложении
        """
        relations = []
        
        # Паттерн-based extraction для функциональных отношений
        for pattern in self.functional_patterns:
            matches = re.finditer(pattern, sentence, re.IGNORECASE)
            for match in matches:
                subject, relation, obj = match.groups()
                
                # Проверяем, что субъект и объект являются извлеченными сущностями
                if self._is_valid_entity_pair(subject, obj, entities):
                    relations.append({
                        'subject': subject,
                        'relation': relation,
                        'object': obj,
                        'type': 'FUNCTIONAL',
                        'confidence': 0.7,
                        'sentence': sentence
                    })
        
        # Паттерн-based extraction для иерархических отношений
        for pattern in self.hierarchical_patterns:
            matches = re.finditer(pattern, sentence, re.IGNORECASE)
            for match in matches:
                groups = match.groups()
                # Обрабатываем различные структуры паттернов
                if len(groups) >= 3:
                    subject = groups[0]
                    relation = groups[-2] if groups[-2] else groups[1]
                    obj = groups[-1]
                    
                    if self._is_valid_entity_pair(subject, obj, entities):
                        relations.append({
                            'subject': subject,
                            'relation': relation,
                            'object': obj,
                            'type': 'HIERARCHICAL',
                            'confidence': 0.7,
                            'sentence': sentence
                        })
        
        return relations
    
    def _is_valid_entity_pair(self, subject: str, obj: str, entities: List[Dict]) -> bool:
        """
        Проверяет, что субъект и объект являются валидными сущностями.
        
        Args:
            subject: Субъект отношения
            obj: Объект отношения
            entities: Список сущностей
            
        Returns:
            bool: True если пара валидна
        """
        entity_words = [entity['word'].lower() for entity in entities]
        return (subject.lower() in entity_words and 
                obj.lower() in entity_words and 
                subject.lower() != obj.lower())


class KnowledgeGraphBuilder:
    """
    Компонент для построения Knowledge Graph с использованием NetworkX.
    """
    
    def __init__(self):
        self.graph = nx.MultiDiGraph()
        self.entity_counter = Counter()
        self.relation_counter = Counter()
    
    def build_knowledge_graph(self, entities: List[Dict], relations: List[Dict]) -> Dict[str, Any]:
        """
        Строит knowledge graph из сущностей и отношений.
        
        Args:
            entities: Список сущностей
            relations: Список отношений
            
        Returns:
            Dict: Результат построения графа
        """
        self.graph.clear()
        self.entity_counter.clear()
        self.relation_counter.clear()
        
        # Добавляем узлы (сущности)
        self._add_entities_as_nodes(entities)
        
        # Добавляем рёбра (отношения)
        self._add_relations_as_edges(relations)
        
        # Вычисляем статистики
        stats = self._compute_graph_statistics()
        
        # Генерируем JSON представление
        graph_json = self._generate_json_representation()
        
        logger.info(f"Built knowledge graph with {len(self.graph.nodes)} nodes and {len(self.graph.edges)} edges")
        
        return {
            'graph': self.graph,
            'statistics': stats,
            'json_representation': graph_json,
            'nodes_count': len(self.graph.nodes),
            'edges_count': len(self.graph.edges)
        }
    
    def _add_entities_as_nodes(self, entities: List[Dict]):
        """Добавляет сущности как узлы графа."""
        for entity in entities:
            entity_name = entity['word']
            entity_type = entity.get('entity_group', 'UNKNOWN')
            confidence = entity.get('score', 0.0)
            
            # Нормализуем имя сущности
            normalized_name = entity_name.lower().strip()
            
            if normalized_name and len(normalized_name) > 2:
                self.entity_counter[normalized_name] += 1
                
                # Добавляем или обновляем узел
                if self.graph.has_node(normalized_name):
                    # Обновляем атрибуты существующего узла
                    current_confidence = self.graph.nodes[normalized_name].get('confidence', 0.0)
                    self.graph.nodes[normalized_name]['confidence'] = max(current_confidence, confidence)
                    self.graph.nodes[normalized_name]['frequency'] = self.entity_counter[normalized_name]
                else:
                    # Добавляем новый узел
                    self.graph.add_node(
                        normalized_name,
                        entity_type=entity_type,
                        confidence=confidence,
                        frequency=1,
                        original_word=entity_name
                    )
    
    def _add_relations_as_edges(self, relations: List[Dict]):
        """Добавляет отношения как рёбра графа."""
        for relation in relations:
            subject = relation['subject'].lower().strip()
            obj = relation['object'].lower().strip()
            relation_type = relation.get('type', 'UNKNOWN')
            relation_name = relation.get('relation', 'related_to')
            confidence = relation.get('confidence', 0.0)
            
            # Проверяем, что узлы существуют в графе
            if (self.graph.has_node(subject) and self.graph.has_node(obj) and 
                subject != obj):
                
                self.relation_counter[relation_name] += 1
                
                # Добавляем ребро
                self.graph.add_edge(
                    subject,
                    obj,
                    relation=relation_name,
                    relation_type=relation_type,
                    confidence=confidence,
                    frequency=self.relation_counter[relation_name]
                )
    
    def _compute_graph_statistics(self) -> Dict[str, Any]:
        """Вычисляет статистики графа."""
        if len(self.graph.nodes) == 0:
            return {'error': 'Empty graph'}
        
        # Базовые статистики
        stats = {
            'nodes_count': len(self.graph.nodes),
            'edges_count': len(self.graph.edges),
            'density': nx.density(self.graph),
            'is_connected': nx.is_weakly_connected(self.graph)
        }
        
        # Статистики по типам узлов
        node_types = {}
        for node, attrs in self.graph.nodes(data=True):
            node_type = attrs.get('entity_type', 'UNKNOWN')
            node_types[node_type] = node_types.get(node_type, 0) + 1
        stats['node_types'] = node_types
        
        # Статистики по типам отношений
        relation_types = {}
        for _, _, attrs in self.graph.edges(data=True):
            rel_type = attrs.get('relation_type', 'UNKNOWN')
            relation_types[rel_type] = relation_types.get(rel_type, 0) + 1
        stats['relation_types'] = relation_types
        
        # Центральность узлов (топ-10)
        try:
            centrality = nx.degree_centrality(self.graph)
            top_central_nodes = sorted(centrality.items(), key=lambda x: x[1], reverse=True)[:10]
            stats['top_central_nodes'] = top_central_nodes
        except:
            stats['top_central_nodes'] = []
        
        return stats
    
    def _generate_json_representation(self) -> Dict[str, Any]:
        """Генерирует JSON представление графа для веб-интерфейса."""
        # Узлы
        nodes = []
        for node, attrs in self.graph.nodes(data=True):
            nodes.append({
                'id': node,
                'label': attrs.get('original_word', node),
                'type': attrs.get('entity_type', 'UNKNOWN'),
                'confidence': attrs.get('confidence', 0.0),
                'frequency': attrs.get('frequency', 1)
            })
        
        # Рёбра
        edges = []
        for source, target, attrs in self.graph.edges(data=True):
            edges.append({
                'source': source,
                'target': target,
                'relation': attrs.get('relation', 'related_to'),
                'type': attrs.get('relation_type', 'UNKNOWN'),
                'confidence': attrs.get('confidence', 0.0),
                'frequency': attrs.get('frequency', 1)
            })
        
        return {
            'nodes': nodes,
            'edges': edges,
            'metadata': {
                'created_at': pd.Timestamp.now().isoformat(),
                'total_nodes': len(nodes),
                'total_edges': len(edges)
            }
        }


class BaselineKnowledgeGraphPipeline:
    """
    Основной класс для запуска baseline модели построения Knowledge Graph.
    """
    
    def __init__(self, config: Dict[str, Any] = None):
        self.config = config or {}
        
        # Инициализируем компоненты
        self.pdf_extractor = PDFTextExtractor()
        self.text_preprocessor = TextPreprocessor()
        self.entity_extractor = BERTEntityExtractor()
        self.relation_extractor = BiLSTMCRFRelationExtractor()
        self.graph_builder = KnowledgeGraphBuilder()
        
        logger.info("Initialized BaselineKnowledgeGraphPipeline")
    
    def process_pdf_document(self, pdf_path: str) -> Dict[str, Any]:
        """
        Обрабатывает один PDF документ и строит Knowledge Graph.
        
        Args:
            pdf_path: Путь к PDF файлу
            
        Returns:
            Dict: Результат обработки с построенным графом
        """
        logger.info(f"Processing PDF document: {pdf_path}")
        
        try:
            # 1. Извлечение текста из PDF
            raw_text = self.pdf_extractor.extract_text_from_pdf(pdf_path)
            if not raw_text.strip():
                return {'error': f'No text extracted from {pdf_path}'}
            
            # 2. Предварительная обработка текста
            preprocessed_data = self.text_preprocessor.preprocess_text(raw_text)
            sentences = preprocessed_data['sentences']
            
            if not sentences:
                return {'error': f'No valid sentences found in {pdf_path}'}
            
            # 3. Извлечение сущностей
            entities = self.entity_extractor.extract_entities(sentences)
            
            if not entities:
                return {'error': f'No entities extracted from {pdf_path}'}
            
            # 4. Извлечение отношений
            relations = self.relation_extractor.extract_relations(sentences, entities)
            
            # 5. Построение Knowledge Graph
            kg_result = self.graph_builder.build_knowledge_graph(entities, relations)
            
            return {
                'success': True,
                'pdf_path': pdf_path,
                'text_length': len(raw_text),
                'sentences_count': len(sentences),
                'entities_count': len(entities),
                'relations_count': len(relations),
                'knowledge_graph': kg_result,
                'processing_steps': {
                    'text_extraction': 'completed',
                    'preprocessing': 'completed',
                    'entity_extraction': 'completed',
                    'relation_extraction': 'completed',
                    'graph_building': 'completed'
                }
            }
            
        except Exception as e:
            logger.error(f"Error processing {pdf_path}: {str(e)}")
            return {'error': f'Processing failed for {pdf_path}: {str(e)}'}
    
    def process_pdf_corpus(self, pdf_directory: str) -> Dict[str, Any]:
        """
        Обрабатывает корпус PDF документов.
        
        Args:
            pdf_directory: Директория с PDF файлами
            
        Returns:
            Dict: Результаты обработки всего корпуса
        """
        pdf_files = glob.glob(os.path.join(pdf_directory, "**/*.pdf"), recursive=True)
        logger.info(f"Found {len(pdf_files)} PDF files in {pdf_directory}")
        
        results = []
        successful_processing = 0
        
        for pdf_file in pdf_files:
            result = self.process_pdf_document(pdf_file)
            results.append(result)
            
            if result.get('success', False):
                successful_processing += 1
        
        return {
            'total_files': len(pdf_files),
            'successful_processing': successful_processing,
            'success_rate': successful_processing / len(pdf_files) if pdf_files else 0,
            'individual_results': results
        }
    
    def save_results(self, results: Dict[str, Any], output_path: str):
        """
        Сохраняет результаты обработки в JSON файл.
        
        Args:
            results: Результаты обработки
            output_path: Путь для сохранения
        """
        try:
            # Преобразуем NetworkX граф в сериализуемый формат
            serializable_results = self._make_json_serializable(results)
            
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(serializable_results, f, indent=2, ensure_ascii=False)
            
            logger.info(f"Results saved to {output_path}")
            
        except Exception as e:
            logger.error(f"Error saving results: {str(e)}")
    
    def _make_json_serializable(self, obj):
        """Делает объект сериализуемым в JSON."""
        if isinstance(obj, dict):
            result = {}
            for key, value in obj.items():
                if key == 'graph' and hasattr(value, 'nodes'):
                    # Пропускаем NetworkX объект, оставляем только JSON представление
                    continue
                else:
                    result[key] = self._make_json_serializable(value)
            return result
        elif isinstance(obj, list):
            return [self._make_json_serializable(item) for item in obj]
        elif isinstance(obj, (np.integer, np.floating)):
            return obj.item()
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return obj


# Пример использования
def main():
    """Пример запуска baseline модели."""
    
    # Конфигурация
    config = {
        'pdf_directory': '../data',  # Директория с PDF файлами
        'output_path': 'knowledge_graph_results.json',
        'bert_model': 'dbmdz/bert-large-cased-finetuned-conll03-english'
    }
    
    # Инициализация pipeline
    pipeline = BaselineKnowledgeGraphPipeline(config)
    
    # Обработка одного файла (для тестирования)
    # result = pipeline.process_pdf_document('path/to/test.pdf')
    # print(json.dumps(result, indent=2))
    
    # Обработка всего корпуса
    # corpus_results = pipeline.process_pdf_corpus(config['pdf_directory'])
    # pipeline.save_results(corpus_results, config['output_path'])
    
    print("Baseline модель готова к использованию!")
    print("\nОсновные компоненты:")
    print("1. PDFTextExtractor - извлечение текста из PDF")
    print("2. TextPreprocessor - обработка текста с NLTK")
    print("3. BERTEntityExtractor - извлечение сущностей с BERT")
    print("4. BiLSTMCRFRelationExtractor - извлечение отношений")
    print("5. KnowledgeGraphBuilder - построение графа с NetworkX")


if __name__ == "__main__":
    main()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pozoy\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pozoy\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\pozoy\AppData\Roaming\nltk_data...
ERROR:__main__:Error loading BERT model: name 'torch' is not defined
INFO:__main__:Initialized BaselineKnowledgeGraphPipeline


Baseline модель готова к использованию!

Основные компоненты:
1. PDFTextExtractor - извлечение текста из PDF
2. TextPreprocessor - обработка текста с NLTK
3. BERTEntityExtractor - извлечение сущностей с BERT
4. BiLSTMCRFRelationExtractor - извлечение отношений
5. KnowledgeGraphBuilder - построение графа с NetworkX


# Тест модели


In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')



"""
baseline_pdf_nospacy.py
--------------------------------------------------
Минимальный, но рабочий конвейер:
1. Поиск PDF-файлов
2. Извлечение текста (PyMuPDF)
3. Предобработка (NLP-минимум на NLTK)
4. Частотный анализ + простая визуализация
5. Тематическое моделирование LDA (gensim)
6. Сохранение результатов
--------------------------------------------------
Требуемые библиотеки:
    pip install pymupdf nltk pandas numpy tqdm gensim pyldavis wordcloud seaborn matplotlib
"""

import os
import glob
import re
import itertools
import collections
from pathlib import Path
from pprint import pprint

import fitz                    # PyMuPDF
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

# --- NLTK: токенизация, лемматизация, стоп-слова
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


# --- Анализ/моделирование
from gensim import corpora, models
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis


# ------------------------------ #
#   1. ПОДГОТОВКА NLTK-РЕСУРСОВ  #
# ------------------------------ #
nltk_needed = ['punkt', 'wordnet', 'stopwords']
for res in nltk_needed:
    try:
        nltk.data.find(f'tokenizers/{res}') if res == 'punkt' else nltk.data.find(f'corpora/{res}')
    except LookupError:
        nltk.download(res)

lemmatizer = WordNetLemmatizer()
stop_en = set(stopwords.words('english'))
# при желании добавьте stopwords.words('russian')

# ------------------------------ #
#   2. ПОИСК И ЗАГРУЗКА PDF      #
# ------------------------------ #
PDF_DIR = Path('../data')           # <-- укажите свою папку
PDF_FILES = list(PDF_DIR.rglob('*.pdf'))
print(f'Найдено файлов: {len(PDF_FILES)}')

def extract_text(path: Path) -> str:
    text_chunks = []
    try:
        doc = fitz.open(path)
        for page in doc:
            text_chunks.append(page.get_text('text'))
    except Exception as e:
        print(f'[!] {path.name}: {e}')
    return '\n'.join(text_chunks)

doc_texts = []
for pdf in tqdm(PDF_FILES, desc='Чтение PDF'):
    doc_texts.append(extract_text(pdf))

# ------------------------------ #
#   3. NLP-ПРЕДОБРАБОТКА         #
# ------------------------------ #
def clean_token(t: str) -> str:
    t = t.lower()
    t = re.sub(r'[^a-zа-яё]+', '', t)  # оставляем только буквы
    return t

def tokenize(text: str):
    tokens = nltk.word_tokenize(text)
    cleaned = []
    for t in tokens:
        t = clean_token(t)
        if len(t) < 3 or t in stop_en:
            continue
        lemma = lemmatizer.lemmatize(t)
        cleaned.append(lemma)
    return cleaned

token_docs = [tokenize(t) for t in tqdm(doc_texts, desc='Токенизация')]

# ------------------------------ #
#   4. ЧАСТОТНЫЙ АНАЛИЗ          #
# ------------------------------ #
all_tokens = list(itertools.chain.from_iterable(token_docs))
freqs = collections.Counter(all_tokens)
print('\nТоп-20 слов:')
pprint(freqs.most_common(20))

# ------------------------------ #
#   5. LDA-ТЕМАТИЧЕСКАЯ МОДЕЛЬ   #
# ------------------------------ #
dictionary = corpora.Dictionary(token_docs)
bow_corpus = [dictionary.doc2bow(text) for text in token_docs]

NUM_TOPICS = 10
lda = models.LdaModel(bow_corpus,
                      num_topics=NUM_TOPICS,
                      id2word=dictionary,
                      passes=10,
                      random_state=42)

print('\nТемы LDA:')
for i, topic in lda.print_topics(num_topics=NUM_TOPICS, num_words=10):
    print(f'Topic #{i}: {topic}')

# ------------------------------ #
#   6. ВИЗУАЛИЗАЦИЯ (PyLDAvis)   #
# ------------------------------ #
vis_data = gensimvis.prepare(lda, bow_corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(vis_data, 'lda_vis.html')
print('Визуализация сохранена: lda_vis.html')

# ------------------------------ #
#   7. СОХРАНЕНИЕ РЕЗУЛЬТАТОВ    #
# ------------------------------ #
df = pd.DataFrame({
    'file': [p.name for p in PDF_FILES],
    'text': doc_texts,
    'tokens': token_docs
})
df.to_pickle('pdf_corpus.pkl')
print('Corpus saved -> pdf_corpus.pkl')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pozoy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pozoy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Найдено файлов: 10


Токенизация:   0%|          | 0/10 [00:00<?, ?it/s]


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\pozoy/nltk_data'
    - 'c:\\Users\\pozoy\\Desktop\\smd\\.venv\\nltk_data'
    - 'c:\\Users\\pozoy\\Desktop\\smd\\.venv\\share\\nltk_data'
    - 'c:\\Users\\pozoy\\Desktop\\smd\\.venv\\lib\\nltk_data'
    - 'C:\\Users\\pozoy\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
